In [14]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from aquacrop.utils import prepare_weather, get_filepath
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
#from aquacrop.entities import IrrigationManagement
from os import chdir, getcwd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import csv
import pickle 
#from dfply import *

from os import chdir, getcwd
import os 
import datetime
import shapefile as shp
import pandas as pd
import geopandas as gpd
from shapely import geometry
import math
from os import listdir
from os.path import isfile, join
import glob

In [ ]:
os.chdir('/home/jupyter-wndlovu/') # change working directory

wd=getcwd()
#wd

In [36]:
# add county files
# 
def fileInput(filepath):
    """This function is used to read input data (.csv files) stored in folders"""
    path =  filepath# landouse folder path
    files = [f for f in listdir(path) if isfile(join(path, f))] # read files from folder
    dfs_list = []  # List to store the dataframes

    for file in files: # read files and save them a list of dataframes
        file_path = os.path.join(path, file)
        df = pd.read_csv(file_path) 
        dfs_list.append(df)
        
    return(dfs_list)


In [43]:
gridmet_list = fileInput(wd + '/eggs/gmd4_gridMET')
lai_list = fileInput(wd + '/eggs/leaf_area_index') 
soils_list = fileInput(wd + '/eggs/gmd4_soils_county')


In [74]:
# clean the datasets by the 
gridmet_df = pd.concat(gridmet_list, ignore_index = True)
                         
                    
gridmet_df = gridmet_df.assign(tmmn = gridmet_df.tmmn-273.15, # convert to celcius
                    tmmx = gridmet_df.tmmx-273.15,
                    date_ymd = pd.to_datetime(gridmet_df['date_ymd'], format='%Y%m%d'))


# rename variables
gridmet_df  = gridmet_df.rename(columns = {
                                'tmmn':'MinTemp',
                                'tmmx':'Maxtemp',
                                'pr':'Precipitation',
                                'eto':'ReferenceET',
                                'date_ymd':'Date'
                                })

gridmet_df = gridmet_df[['crop_mn_codeyear',
                         'MinTemp', 
                         'Maxtemp',
                         'Precipitation', 
                         'ReferenceET',
                         'Date'
                        ]]

In [75]:
group_gridmet = gridmet_df.groupby('crop_mn_codeyear')

# list to store the dfs
gridmet_county_id = []

# create separate dfs
for i, j in group_gridmet:
    gridmet_county_id.append(j.copy())
    
print(len(gridmet_county_id))

In [87]:
lai_df = pd.concat(lai_list, ignore_index = True)

lai_df = lai_df.assign(cc = (100.5*(1-np.exp(-0.6*lai_df['Lai']))**1.2),
                      date = pd.to_datetime(lai_df['date_ymd'], format='%Y%m%d')) # calc canopy cover Hsiao et al. (2009)

lai_df = lai_df[['crop_mn_codeyear', 'date', 'Lai', 'cc']]

In [92]:
group_lai = lai_df.groupby('crop_mn_codeyear')

# list to store the dfs
lai_county_id = []

# create separate dfs
for i, j in group_lai:
    lai_county_id.append(j.copy())